# Single-point protein mutations with the OpenFE ecosystem

The purpose of this notebook is to showcase the capabilities of computing relative binding free energy effects from single-point protein mutations on the ABL1 protein in complex with the imatinib inhibitor. The structures and mutation data are taken from Singh, et al. at https://pubs.acs.org/doi/full/10.1021/acs.jpcb.4c07794

![](abl1_imatinib.png)

## Generating Mutated Structures

First we need to generate the mutated structures we want to study. For this tutorial we will focus on the `L364I` (Leucine to Isoleucine) mutation.

To generate the mutated structures we will be using the `pdbfixer` software tool from the OpenMM ecosystem. This will generate a PDB-formatted file with the mutated structure of the protein as an output. We use OpenMM's `PDBFile` to write the PDB file.

In [1]:
# Input wild type protein mutation
input_pdb = "inputs/abl1_structure.pdb"
# mutation specification
wildtype_res = "LEU"
mutated_res = "ILE"
residue_number = 364
chain_id = "A"

In [2]:
import os
import pdbfixer
from openmm.app import PDBFile
pdb_fixer = pdbfixer.PDBFixer(filename=input_pdb)  # read input pdb
# Apply mutation
pdb_fixer.applyMutations(mutations=[f"{wildtype_res}-{residue_number}-{mutated_res}"], chain_id=chain_id)
pdb_fixer.findMissingResidues()
pdb_fixer.findMissingAtoms()
pdb_fixer.addMissingAtoms()
pdb_fixer.addMissingHydrogens()
omm_topology = pdb_fixer.topology
omm_positions = pdb_fixer.positions
mutant_out_dir = "./mutant_structures"
os.makedirs(mutant_out_dir, exist_ok=True)
with open(f"{mutant_out_dir}/abl1_mutated.pdb", "w") as out_file:
    PDBFile.writeFile(omm_topology, omm_positions, out_file, keepIds=True)

## Defining Atom Mappings

To define mappings between the wild type and mutated proteins, we will be using the [Kartograf](https://github.com/OpenFreeEnergy/kartograf/) mapping software. For that we need to create OpenFE `ProteinComponent`s from the PDB files and perform the mapping. 

In [3]:
from openfe import ProteinComponent
wt_component = ProteinComponent.from_pdb_file("inputs/abl1_structure.pdb")
mut_component = ProteinComponent.from_pdb_file("mutant_structures/abl1_mutated.pdb")

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


In [4]:
from kartograf import KartografAtomMapper
atom_mapper = KartografAtomMapper()  # default settings should work for our purposes
mapping = next(atom_mapper.suggest_mappings(wt_component, mut_component))

Here we can inspect the mapping object to notice that the whole protein should be mapped except the side chain atoms in the mutated residue that do not overlap (either by atom type or position).

In [5]:
len(mapping.componentA_to_componentB)

4302

We can query for unique atom indices and types in the mapping components to verify we get the expected mapping results

In [6]:
print("Unique atoms in component A:")
for atom_idx in mapping.componentA_unique:
    print(f"\tAtom type: {mapping.componentA.to_rdkit().GetAtomWithIdx(atom_idx).GetSymbol()}, index {atom_idx}")
print("Unique atoms in component B:")
for atom_idx in mapping.componentB_unique:
    print(f"\tAtom type: {mapping.componentB.to_rdkit().GetAtomWithIdx(atom_idx).GetSymbol()}, index {atom_idx}")

Unique atoms in component A:
	Atom type: C, index 2170
	Atom type: C, index 2171
	Atom type: C, index 2172
	Atom type: H, index 2175
	Atom type: H, index 2177
	Atom type: H, index 2178
	Atom type: H, index 2179
	Atom type: H, index 2180
	Atom type: H, index 2181
	Atom type: H, index 2182
	Atom type: H, index 2183
Unique atoms in component B:
	Atom type: C, index 2173
	Atom type: H, index 2174
	Atom type: H, index 2175
	Atom type: H, index 2176
	Atom type: C, index 2177
	Atom type: H, index 2178
	Atom type: H, index 2179
	Atom type: H, index 2180
	Atom type: C, index 2181
	Atom type: H, index 2182
	Atom type: H, index 2183


<div class="alert alert-block alert-info">
⚠️ Better mapping visualization tools for protein mutations are being worked on and will be available in the future.
</div>


Lastly for this part we want to store the generated mappings in static files that we can use in the future as reference or to extract the information up to this step. We use the OpenFE JSON serialization capabilities for this

In [7]:
# Serializing mappings
os.makedirs("./mappings", exist_ok=True)
with open(f"mappings/abl1_example_mapping.json", "w") as out_file:
    mapping.to_json(out_file)

/home/ijpulidos/miniforge3/envs/feflow-dev/lib/python3.12/site-packages/gufe/utils.py:32: UserWarning: mode='w' specified with TextIOWrapper. User-specified mode will be ignored.
  warnings.warn(


## Defining Chemical Systems

Now that we have mutated structures and the mappings between them, we can define the systems we want to run in our simulation. This information is encoded in the `ChemicalSystem` object from the OpenFE toolset. Chemical systems are sets of components, so we first want to define our components.

In [8]:
from openfe import SmallMoleculeComponent,SolventComponent
from openff.units import unit

# Read protein (wt and ut) components from PDBs
wt_protein = ProteinComponent.from_pdb_file("inputs/abl1_structure.pdb")
mut_protein = ProteinComponent.from_pdb_file("mutant_structures/abl1_mutated.pdb")
# Read ligand from sdf file
ligand = SmallMoleculeComponent.from_sdf_file("inputs/imatinib_ligand.sdf")
# Specify solvent component
solvent = SolventComponent(positive_ion="Na", negative_ion="Cl", neutralize=True, ion_concentration=0.15*unit.molar)

We will be needing a total of four(4) chemical systems, two for the complex leg of the thermodynamic cycle (ABL1+Imatinib) and other two for the solvent leg of it (ABL1 alone in solvent).

In [9]:
from openfe import ChemicalSystem
wt_solvent = ChemicalSystem({"protein": wt_protein, "solvent": solvent})
wt_complex = ChemicalSystem({"protein": wt_protein, "ligand": ligand, "solvent": solvent})
mut_solvent = ChemicalSystem({"protein": mut_protein, "solvent": solvent})
mut_complex = ChemicalSystem({"protein": mut_protein, "ligand": ligand, "solvent": solvent})

## Defining simulation settings

To get a relative binding free energy estimate from this mutation we would be performing a nonequilibrium transformation using the `NonEquilibriumCycling` protocol from the [`feflow`](https://github.com/OpenFreeEnergy/feflow) package. To date, this is the only protocol that supports handling protein mutations, but extending the support for other protocols (replica exchange, nonequilibrium switching, and others) is being worked on.

The protocol default settings should give a good starting point to be used to built upon and adapt to our needs, that way we don't need to specify every single parameter/setting, but only the ones that we want to manually control.

In [10]:
from feflow.protocols import NonEquilibriumCyclingProtocol
from openff.units import unit

# global timestep to use in the simulation
timestep = 4 * unit.femtosecond

# Solvent settings
solvent_rbfe_settings = NonEquilibriumCyclingProtocol.default_settings()
solvent_rbfe_settings.integrator_settings.timestep = timestep
solvent_rbfe_settings.integrator_settings.equilibrium_steps = 2500 # Number of timesteps to spend on equilibrium portion
solvent_rbfe_settings.integrator_settings.nonequilibrium_steps = 2500 # Number of timesteps to spend on nonequilibrium portion
solvent_rbfe_settings.num_cycles = 1  # Number of cycles to run
solvent_rbfe_settings.engine_settings.compute_platform = None  # default is to look for CUDA, none allows falling back to OpenCL or CPU
# Complex settings
complex_rbfe_settings = NonEquilibriumCyclingProtocol.default_settings()
complex_rbfe_settings.integrator_settings.timestep = timestep
complex_rbfe_settings.integrator_settings.equilibrium_steps = 2500 # Number of timesteps to spend on equilibrium portion
complex_rbfe_settings.integrator_settings.nonequilibrium_steps = 2500 # Number of timesteps to spend on nonequilibrium portion
complex_rbfe_settings.num_cycles = 1  # Number of cycles to run
complex_rbfe_settings.solvation_settings.solvent_padding = 1 * unit.nanometer
complex_rbfe_settings.engine_settings.compute_platform = None  # default is to look for CUDA, none allows falling back to OpenCL or CPU

/home/ijpulidos/miniforge3/envs/feflow-dev/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/ijpulidos/miniforge3/envs/feflow-dev/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


We can also visually inspect all the settings as a python object/dictionary, as follows:

In [11]:
complex_rbfe_settings

{'alchemical_settings': {'endstate_dispersion_correction': False,
                         'explicit_charge_correction': False,
                         'explicit_charge_correction_cutoff': {'unit': 'nanometer',
                                                               'val': 0.8},
                         'softcore_LJ': 'gapsys',
                         'softcore_alpha': 0.85,
                         'turn_off_core_unique_exceptions': False,
                         'use_dispersion_correction': False},
 'atom_selection_expression': 'not water',
 'engine_settings': {'compute_platform': None, 'gpu_device_index': None},
 'forcefield_cache': 'db.json',
 'forcefield_settings': {'constraints': 'hbonds',
                         'forcefields': ['amber/ff14SB.xml',
                                         'amber/tip3p_standard.xml',
                                         'amber/tip3p_HFE_multivalent.xml',
                                         'amber/phosaa10.xml'],
               

## Defining protocol and transformations

Now that we have our settings defined, we can proceed to apply these settings to our actual protocol, and define what transformations to run in out simulations, that is, what chemical systems will be simulation with our defined protocol.

In [12]:
# Apply settings to protocol
solvent_rbfe_protocol = NonEquilibriumCyclingProtocol(
    settings=solvent_rbfe_settings
)
complex_rbfe_protocol = NonEquilibriumCyclingProtocol(
    settings=complex_rbfe_settings
)

In [13]:
from openfe import Transformation
# Define transformations to run
transformation_complex = Transformation(
            stateA=wt_complex,
            stateB=mut_complex,
            mapping=mapping,
            protocol=complex_rbfe_protocol,  # use complex protocol created above
            name=f"{wildtype_res}-{residue_number}-{mutated_res}_complex"
        )
transformation_solvent = Transformation(
            stateA=wt_solvent,
            stateB=mut_solvent,
            mapping=mapping,
            protocol=solvent_rbfe_protocol,  # use solvent protocol created above
            name=f"{wildtype_res}-{residue_number}-{mutated_res}_solvent"
        )

Now we can store our transformation objects into a JSON file, which have all the information of our simulations and can be ported to other machines/devices for execution.

In [14]:
import pathlib
# first we create the directory
transformation_dir = pathlib.Path("transformation_objects")
transformation_dir.mkdir(exist_ok=True)

# then we write out the transformations
transformation_complex.to_json(transformation_dir / f"{transformation_complex.name}.json")
transformation_solvent.to_json(transformation_dir / f"{transformation_solvent.name}.json")

## Executing the simulation

There are two ways to execute the simulation. One is using the Python API and the second one is using the CLI. 

### Using the python API

Using the python API means we need to create the directed acyclic graphs (DAG) from the protocol and execute them. Please refer to the [underlying documentation on protocols for more information](https://gufe.openfree.energy/en/latest/concepts/included_models.html#protocol).

In [ ]:
# Creating the DAGs
complex_dag = transformation_complex.create()
solvent_dag = transformation_solvent.create()

After creating the DAGs, we can then execute them with the help of utility functions

In [ ]:
from openfe import execute_DAG
# Finally we can run the simulations
complex_path = pathlib.Path('./complex')
complex_path.mkdir(exist_ok=True)

# First the complex transformation
complex_dag_results = execute_DAG(complex_dag, scratch_basedir=complex_path, shared_basedir=complex_path)

# Next the solvent state transformation
solvent_path = pathlib.Path('./solvent')
solvent_path.mkdir(exist_ok=True)

solvent_dag_results = execute_DAG(solvent_dag, scratch_basedir=solvent_path, shared_basedir=solvent_path)

# Get the complex and solvent results
complex_results = complex_rbfe_protocol.gather([complex_dag_results])
solvent_results = solvent_rbfe_protocol.gather([solvent_dag_results])

print(f"Complex dG: {complex_results.get_estimate()}, err {complex_results.get_uncertainty()}")
print(f"Solvent dG: {solvent_results.get_estimate()}, err {solvent_results.get_uncertainty()}")

### Using the `openfe` CLI

Now we can just simply run our simulations using the convenient CLI from `openfe`. In this case we are just executing the complex leg of our simulation for illustrative purposes. Executing the solvent leg is analogous.

In [ ]:
!openfe quickrun transformation_objects/LEU-364-ILE_complex.json -o results_complex.json -d working_dir

In [ ]:
!openfe quickrun transformation_objects/LEU-364-ILE_solvent.json -o results_solvent.json -d working_dir

## Analysis of results

For the analysis of the results we will focus on using the Python API to gather the results from the multiple executions of the protocol

In [ ]:
# Get the complex and solvent results
complex_results = complex_rbfe_protocol.gather([complex_dag_results])
solvent_results = solvent_rbfe_protocol.gather([solvent_dag_results])

print(f"Complex dG: {complex_results.get_estimate()}, err {complex_results.get_uncertainty()}")
print(f"Solvent dG: {solvent_results.get_estimate()}, err {solvent_results.get_uncertainty()}")